In [1]:
import pandas as pd 
import numpy as np
import csv
import baseline_functions as base

# restore saved variables
%store -r summary_F6_fl_model

no stored variable summary_F6_fl_model


In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_F_six_month'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_F_six_month'].values

### Model

In [3]:
#### Logistic
c = [1e-5, 1e-4, 1e-3, 1e-2]
logistic_summary = base.Logistic(FL_X, FL_Y, KY_X, KY_Y,C=c,seed=816)

#### Lasso
c = [1e-5, 1e-4, 1e-3, 1e-2]
lasso_summary = base.Lasso(FL_X, FL_Y, KY_X, KY_Y,C=c,seed=816)

#### LinearSVM
c = [1e-5, 1e-4]
svm_summary = base.LinearSVM(FL_X, FL_Y, KY_X, KY_Y,C=c,seed=816)

#### Random Forest
n_estimators =  [60,80,100]
depth = [1,2]
impurity = [0.001, 0.003]
rf_summary = base.RF(FL_X, FL_Y, KY_X, KY_Y, 
                     depth=depth, 
                     estimators=n_estimators, 
                     impurity=impurity,
                     seed=816)

#### XGBoost
learning_rate = [0.05, 0.07]
depth = [2,3]
n_estimators = [20,40,60]
gamma = [6,8,10]
child_weight = [6,8,10]
subsample = [0.3, 0.5]
xgb_summary = base.XGB(FL_X, FL_Y, KY_X, KY_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       gamma=gamma, 
                       child_weight=child_weight, 
                       subsample=subsample, 
                       seed=816)

#### save results
summary_F6_fl_model = {"Logistic": logistic_summary,
                                    "Lasso": lasso_summary,
                                    "LinearSVM": svm_summary,
                                    "RF": rf_summary,
                                    "XGBoost": xgb_summary}
%store summary_F6_fl_model

Stored 'summary_F6_fl_model' (dict)


In [4]:
results = []

for model_name, model_summary in summary_F6_fl_model.items():
    results.append([model_name, 
                    model_summary['best_auc'], 
                    model_summary['auc_diff'], 
                    model_summary['best_param'], 
                    model_summary['KY_score']])
results

[['Logistic',
  0.6233119475331805,
  0.028421547582951012,
  {'C': 0.001},
  0.6219511494707786],
 ['Lasso',
  0.6185607940100006,
  0.009733002015312686,
  {'C': 0.01},
  0.5960405842595025],
 ['LinearSVM',
  0.584137937160592,
  0.015311960984219386,
  {'C': 0.0001},
  0.5384681045608235],
 ['RF',
  0.6201800499523538,
  0.04274963106570284,
  {'max_depth': 1, 'min_impurity_decrease': 0.001, 'n_estimators': 80},
  0.6554102993637743],
 ['XGBoost',
  0.6140159852396085,
  0.04565020230511585,
  {'gamma': 8,
   'learning_rate': 0.07,
   'max_depth': 3,
   'min_child_weight': 6,
   'n_estimators': 40,
   'subsample': 0.5},
  0.563649764615191]]

In [7]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\Six Month\\"
results = [["Felony",             
            round(logistic_summary['KY_score'],3), 
            round(lasso_summary['KY_score'], 3), 
            round(svm_summary['KY_score'],3), 
            round(rf_summary['KY_score'],3), 
            round(xgb_summary['KY_score'],3)]]
with open(path + 'Six Month FL_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)